# Logistic Regression

Refs:
Apply feature selection is a mandatory stuff for logistic regression, via PCA or Variace Threshold and others, worths the reading: 
https://medium.com/@data.science.enthusiast/feature-selection-techniques-forward-backward-wrapper-selection-9587f3c70cfa

In [1]:
import pandas as pd
import numpy as np
from spaceship_titanic import feature_enginnering as fe

In [2]:
df_train = pd.read_csv("data/train.csv", index_col='PassengerId')
df_test = pd.read_csv("data/test.csv", index_col='PassengerId')
df_train.head(2)

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
PassengerId,,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True


# Data Treatment

In [3]:
def treat_dataset(df:pd.DataFrame) -> pd.DataFrame:
    df = df.drop(['Name'],axis=1)
    df = fe.cabin_inputer(df)
    df = fe.feature_inputer(df)
    df = fe.vip_knn_input(df)
    df['0_bills'] = df[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1) == 0
    df = fe.outliers_to_log(df)
    df = fe.dtype_memory_reducer(df)
    return df
def feature_enginnering(df: pd.DataFrame) -> pd.DataFrame:
    df = pd.get_dummies(df,columns=['HomePlanet','Destination','Deck'],drop_first=True)
    return df

def train_test_treatments(df, full_df):
    df['is_alone'] = fe.calculate_groupsize(df.index, full_df.index) == 1
    return df

In [4]:
df_train = treat_dataset(df_train.copy())
df_train = feature_enginnering(df_train)
df_train = train_test_treatments(df_train, pd.concat([df_train,df_test]))
df_train

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,0_bills,...,HomePlanet_Mars,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,is_alone
PassengerId,,,,,,,,,,,,,,,,,,,,,
0001_01,False,39.0,False,0.000000,0.000000,0.000000,0.000000,0.000000,False,True,...,0,0,1,1,0,0,0,0,0,True
0002_01,False,24.0,False,4.700480,2.302585,3.258097,6.309918,3.806662,True,False,...,0,0,1,0,0,0,0,1,0,True
0003_01,False,58.0,True,3.784190,8.182280,0.000000,8.812248,3.912023,False,False,...,0,0,1,0,0,0,0,0,0,False
0003_02,False,33.0,False,0.000000,7.157735,5.918894,8.110728,5.267858,False,False,...,0,0,1,0,0,0,0,0,0,False
0004_01,False,16.0,False,5.717028,4.262680,5.023881,6.338594,1.098612,True,False,...,0,0,1,0,0,0,0,1,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9276_01,False,41.0,True,0.000000,8.827615,0.000000,7.404888,4.317488,False,False,...,0,0,0,0,0,0,0,0,0,True
9278_01,True,18.0,False,0.000000,0.000000,0.000000,0.000000,0.000000,False,True,...,0,1,0,0,0,0,0,0,1,True
9279_01,False,26.0,False,0.000000,0.000000,7.535297,0.693147,0.000000,True,False,...,0,0,1,0,0,0,0,0,1,True


In [5]:
X_train,y_train = df_train.drop('Transported',axis=1), df_train['Transported']

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

## Evaluation

In [7]:
from sklearn.model_selection import cross_validate, KFold
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=10000)

In [14]:
scores = cross_validate(
    model,
    X_train,
    y_train,
    cv=KFold(5, shuffle=True, random_state=42),
    scoring = "accuracy",
    return_train_score=True
)

In [15]:
model.fit(X_train,y_train)

LogisticRegression(max_iter=10000)

In [16]:
results = {
    'train_score_avg':np.mean(scores['train_score']),
    'train_score_std':np.std(scores['train_score']),
    'test_score_avg':np.mean(scores['test_score']),
    'test_score_std':np.std(scores['test_score']),
}
results

{'train_score_avg': 0.7734097603911253,
 'train_score_std': 0.0023323801400104423,
 'test_score_avg': 0.7711968903996914,
 'test_score_std': 0.007575513185981753}

# Submission

In [21]:
X_test = treat_dataset(df_test)
X_test = feature_enginnering(X_test)
X_test = train_test_treatments(X_test, pd.concat([df_train,df_test]))
X_test = pd.DataFrame(scaler.fit_transform(X_test), index=X_test.index, columns=X_test.columns)
X_test.head(2)

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,0_bills,HomePlanet_Europa,HomePlanet_Mars,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,is_alone
PassengerId,,,,,,,,,,,,,,,,,,,,
0013_01,1.330443,-0.114147,-0.132689,-0.64483,-0.646513,-0.628742,-0.661615,-0.624214,1.170830,-0.553131,-0.525314,-0.315862,0.634992,-0.30408,-0.300857,-0.244898,-0.341629,-0.714311,1.490179,0.909823
0018_01,-0.751630,-0.684313,-0.132689,-0.64483,0.140685,-0.628742,2.226792,-0.624214,-0.854095,-0.553131,-0.525314,-0.315862,0.634992,-0.30408,-0.300857,-0.244898,-0.341629,1.399951,-0.671060,0.909823


In [22]:
y_test = model.predict(X_test)

In [25]:
submission = pd.Series(y_test, index=[X_test.index]).to_frame('Transported')
submission.head()

,Transported
PassengerId,
0013_01,True
0018_01,False
0019_01,True
0021_01,True
0023_01,True


In [26]:
submission.to_csv('data/submission_LR_2022-11-05.csv')